## Harnessing instrument simulators for model evaluation

In this 45 minute lecture, we will cover the basics of how to evaluate model simulations using instrument simulators. This course covers:

    1. The basic principles of instrument simulators for models
    2. Examining E3SM single column model data using xarray
    3. Using the Earth Model Column Collaboratory to create simulated radar moments for evaluation
    4. Loading ARM KAZR data at COMBLE for comparison against model simulated radar moments


## Instrument simulators

Models do not typically output parameters that are directly comparable to atmospheric measurements. This complicates the evaluation of model performance using atmospheric observations, which are typically the only metric available for assessing the performance of such simulations. In order to assess how well a model simulation represents the target atmospheric phenomena -- whether it be convection, cloud cover, or aerosols, instrument simulators provide a key bridge between the measurements and model simulations. They do this by simulating the instrument output from within the model simulation. 

## How do instrument simulators work?

This section, in particular, will focus on how instrument simulators for instruments designed to detect clouds and precipitation such as radars and lidars.

Bulk cloud microphysics parameterizationst will typically predict either one or two moments of the hydrometeor size distribution $N(D)$. The "classic" Morrison-Gettleman scheme will predict these two moments (usually number concentration $N$ and mass mixing ratio $Q$) for four hydrometeor categories: cloud liquid (cl), cloud ice (ci), precipitating liquid (pl), and precipitating ice (pi) (Morrison and Gettleman 2008). However, $N$ and $Q$ are not measured by radars and lidars. Instead, radar and lidars first emit a beam of microwave (radar) or near-infrared (lidar) radiation. They then measure the in-phase and quadrature signals of the signal recieved by the transciever that recieves the backscattered signal from the hydrometers in the sample volume. These signals are then converted into Doppler spectra, then subsequently converted into momemts such as radar reflectivity factor $Z_{e}$ and radial velocity $v_{d}$.

The method by which we then have to create an apples-to-apples comparison of the model simulated cloud properties against those observed by the radar is to simulate the radar inside the model simulation. This is done by taking in the assumptions made by the model's radiative transfer scheme with regards to forward scattering. For example, to calculate the reflectivity using Equation (1), we need to know the backscattering cross section of the hydrometeor $\beta_{hyd}$, the wavelength of the radar $\lambda$ and the dielectric constant of the scattering medium (ice or water) $K_{w}$. Then, once we know these, which are all inputs either assumed by the model or are a property of the instrument we are simulating, can calculate $Z_{e}$ as $$Z_{e, hyd} = \frac{\beta_{hyd} \lambda^{4}}{\pi^{5}K_{w}^2}$$

In this section of the course, we will utilize the [Earth Column Model Collaboratory (EMC^2)](https://github.com/columncolab/EMC2) to simulate the Ka-band Scanning Zenith Radar (KAZR) inside a DHARMA LES simulation. EMC^2 is an open source framework that will provide simulated radar moments from single column model (SCM) simulations, utilizing a sub-column generator to generate a distribution of realizations of the SCM column according to the model cloud fraction. This takes into account the potential uncertainty due to a vertically pointing radar only capturing a fraction of the SCM column. The simulated radar or lidar moments from the subcolumns can then be compared against the observed moments from the instrument. For more information on both the calculation of the simulated moments and sub-column generation, see Silber et al. (2022).

In addition, EMC^2 has an LES mode where the model subcolumns are mapped directly to the x and y coordinates of the LES simulation. For this tutorial, we will focus on the LES simulations from DHARMA for a case observed during COMBLE.

# References


Morrison, H., J. A. Milbrandt, G. Bryan, K. Ikeda, S. A. Tessendorf, and G. Thompson, 2015: Parameterization of cloud microphysics based on the prediction of bulk ice particle properties. Part II: Case study comparisons with observations and other schemes. J. Atmos. Sci., 72, 312–339, https://doi.org/10.1175/JAS-D-14-0066.1.

Silber, I., Jackson, R. C., Fridlind, A. M., Ackerman, A. S., Collis, S., Verlinde, J., and Ding, J.: The Earth Model Column Collaboratory (EMC2) v1.1: an open-source ground-based lidar and radar instrument simulator and subcolumn generator for large-scale models, Geosci. Model Dev., 15, 901–927, https://doi.org/10.5194/gmd-15-901-2022, 2022. 


In [2]:
import xarray as xr
import emc2

## Preview DHARMA simulation

In [3]:
ds = xr.open_dataset('../data/comble/dharma/emc2_prep.nc')
ds

<xarray.Dataset>
Dimensions:         (x: 256, y: 256, hgt: 159)
Coordinates:
  * x               (x) float64 -1.275e+04 -1.265e+04 ... 1.265e+04 1.275e+04
  * y               (y) float64 -1.275e+04 -1.265e+04 ... 1.265e+04 1.275e+04
  * hgt             (hgt) float32 10.0 32.5 60.0 ... 6.85e+03 6.92e+03 6.975e+03
Data variables: (12/23)
    z               (hgt, x, y) float32 ...
    u_wind          (hgt, x, y) float32 ...
    v_wind          (hgt, x, y) float32 ...
    w_wind          (hgt, x, y) float32 ...
    qcl             (hgt, x, y) float32 ...
    qci             (hgt, x, y) float32 ...
    ...              ...
    strat_cl_frac   (hgt, x, y) float32 ...
    strat_ci_frac   (hgt, x, y) float32 ...
    strat_pl_frac   (hgt, x, y) float32 ...
    strat_pi_frac   (hgt, x, y) float32 ...
    strat_pir_frac  (hgt, x, y) float32 ...
    conv_dat        (hgt, x, y) float32 ...

In [4]:
ds

<xarray.Dataset>
Dimensions:         (x: 256, y: 256, hgt: 159)
Coordinates:
  * x               (x) float64 -1.275e+04 -1.265e+04 ... 1.265e+04 1.275e+04
  * y               (y) float64 -1.275e+04 -1.265e+04 ... 1.265e+04 1.275e+04
  * hgt             (hgt) float32 10.0 32.5 60.0 ... 6.85e+03 6.92e+03 6.975e+03
Data variables: (12/23)
    z               (hgt, x, y) float32 ...
    u_wind          (hgt, x, y) float32 ...
    v_wind          (hgt, x, y) float32 ...
    w_wind          (hgt, x, y) float32 ...
    qcl             (hgt, x, y) float32 ...
    qci             (hgt, x, y) float32 ...
    ...              ...
    strat_cl_frac   (hgt, x, y) float32 ...
    strat_ci_frac   (hgt, x, y) float32 ...
    strat_pl_frac   (hgt, x, y) float32 ...
    strat_pi_frac   (hgt, x, y) float32 ...
    strat_pir_frac  (hgt, x, y) float32 ...
    conv_dat        (hgt, x, y) float32 ...

In [ ]:
## Domain selection

In order to 

In [ ]:
ds['qcl'].isel(hgt=90).plot()

In [ ]:
ds['qpi'].sel(x=slice(-10000., 5000.), y=slice(-10000, -3000.)).isel(hgt=0).plot()

In [5]:
dharma = emc2.core.model.DHARMA('../data/comble/dharma/emc2_prep.nc', single_pi_class=False,
                               bounding_box=(-10000., -10000., 4000., -3000.))
dharma.ds

../data/comble/dharma/emc2_prep.nc is a regional output dataset; Stacking the time, lat, and lon dims for processing with EMC^2.


<xarray.Dataset>
Dimensions:         (hgt: 159, time_lat_lon: 9800, y_tmp: 70, x_tmp: 140,
                     dom_col_tmp: 1)
Coordinates:
  * hgt             (hgt) float64 10.0 32.5 60.0 ... 6.85e+03 6.92e+03 6.975e+03
  * y_tmp           (y_tmp) float64 -9.95e+03 -9.85e+03 ... -3.15e+03 -3.05e+03
  * x_tmp           (x_tmp) float64 -9.95e+03 -9.85e+03 ... 3.85e+03 3.95e+03
  * dom_col_tmp     (dom_col_tmp) int64 0
  * time_lat_lon    (time_lat_lon) object MultiIndex
  * y               (time_lat_lon) float64 -9.95e+03 -9.95e+03 ... -3.05e+03
  * x               (time_lat_lon) float64 -9.95e+03 -9.85e+03 ... 3.95e+03
  * dom_col         (time_lat_lon) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
Data variables: (12/23)
    z               (time_lat_lon, hgt) float64 10.0 32.5 ... 6.92e+03 6.975e+03
    u_wind          (time_lat_lon, hgt) float64 8.499 8.521 ... 1.356 1.376
    v_wind          (time_lat_lon, hgt) float64 -9.769 -10.85 ... -11.05 -11.05
    w_wind          (time_lat_lon, hgt) float64 0.0 0.06021 ... -0.003778
    qcl             (time_lat_lon, hgt) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    qci             (time_lat_lon, hgt) float64 5.233e-07 5.646e-07 ... 0.0 0.0
    ...              ...
    strat_cl_frac   (time_lat_lon, hgt) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    strat_ci_frac   (time_lat_lon, hgt) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    strat_pl_frac   (time_lat_lon, hgt) float64 0.0 0.0 0.0 1.0 ... 0.0 0.0 0.0
    strat_pi_frac   (time_lat_lon, hgt) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    strat_pir_frac  (time_lat_lon, hgt) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    conv_dat        (time_lat_lon, hgt) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [ ]:
KAZR = emc2.core.instruments.KAZR('nsa')
subcols = emc2.simulator.main.make_simulated_data(dharma, KAZR, 1, use_rad_logic=False)

## Creating subcolumns...
No convective processing for DHARMA
num_subcolumns == 1 (subcolumn generator turned off); setting subcolumns frac fields to 1 for startiform cl and ci based on q > 0. kg/kg
Done! total processing time = 0.03s
num_subcolumns == 1 (subcolumn generator turned off); setting subcolumns frac fields to 1 for strat precip based on q > 0. kg/kg
Done! total processing time = 0.03s
num_subcolumns == 1 (subcolumn generator turned off); setting subcolumns q (and N micro logic) fields for cl equal to grid-cell mean
num_subcolumns == 1 (subcolumn generator turned off); setting subcolumns q (and N micro logic) fields for ci equal to grid-cell mean
num_subcolumns == 1 (subcolumn generator turned off); setting subcolumns q (and N micro logic) fields for pl equal to grid-cell mean
num_subcolumns == 1 (subcolumn generator turned off); setting subcolumns q (and N micro logic) fields for pi equal to grid-cell mean
num_subcolumns == 1 (subcolumn generator turned off); setting subcol